<a href="https://colab.research.google.com/github/hirokimituya/stock-price-analysis/blob/main/technical_analysis/%E9%8A%98%E6%9F%84%E9%81%B8%E5%AE%9A%E9%96%A2%E6%95%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Google Colaboratoryに画像を保存できるように設定
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/')

Mounted at /content/drive


# 銘柄コードから株価情報を取得する関数

In [4]:
!pip install yahoo_finance_api2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for yahoo_finance_api2: filename=yahoo_finance_api2-0.0.12-py3-none-any.whl size=3714 sha256=676bb000417189745f1ed1efb63fb83b501d8b689128d648a0dd4c9cab9057ba
  Stored in directory: /root/.cache/pip/wheels/e2/40/e3/f3d8054a41d7d85065555037525f48f56119cafbc5fcddf427
Successfully built yahoo_finance_api2


In [92]:
from yahoo_finance_api2 import share as yapi2
import datetime as dt
from datetime import date,timedelta
import pandas as pd

# 株価データを取得するメソッド
def get_stock_data_yapi2(code, period_type='year', period=10):
    """株価データを取得する

    :param code: 取得する株価データの銘柄コード
    :param period_type: 取得する株価データの期間の単位（例: 'year', '）
    :param period: 取得する株価データの期間の数値
    :return: 株価データのデータフレーム
    """
    if period_type == 'month':
        period_type_value = yapi2.PERIOD_TYPE_MONTH
    elif period_type == 'week':
        period_type_value = yapi2.PERIOD_TYPE_WEEK
    elif period_type == 'day':
        period_type_value = yapi2.PERIOD_TYPE_DAY
    else:
        period_type_value = yapi2.PERIOD_TYPE_YEAR

    data = yapi2.Share(f'{code}.T').get_historical(
        period_type_value,
        period,
        yapi2.FREQUENCY_TYPE_DAY,
        1
    )

    df = pd.DataFrame(data)

    # タイムスタンプをDateTime型に変更
    df['datetime'] = pd.to_datetime(df['timestamp'], unit='ms')

    # 日本時間へ変換
    df['Date'] = df['datetime'] + dt.timedelta(hours=9)

    # インデックスをDateカラムに設定
    df.index = df['Date'].dt.date

    # 不要なカラムを削除
    del df['Date'], df['datetime'], df['timestamp']

    # 各カラムの最初の文字を大文字に変換
    for col in df.columns:
        df[f'{col.capitalize()}'] = df[col]
        del df[col]

    return df

In [10]:
get_stock_data_yapi2(9501)

,Open,High,Low,Close,Volume
Date,,,,,
2013-05-13,451.0,463.0,436.0,442.0,89462000
2013-05-14,446.0,522.0,443.0,522.0,352213400
2013-05-15,517.0,612.0,504.0,513.0,699968100
2013-05-16,533.0,613.0,526.0,613.0,618650500
2013-05-17,633.0,656.0,607.0,626.0,505085200
...,...,...,...,...,...
2023-05-01,487.0,503.0,484.0,494.0,27854600
2023-05-02,492.0,494.0,486.0,486.0,16652300
2023-05-08,486.0,499.0,485.0,489.0,18300400


# JPX400の銘柄取得

In [93]:
def filter_ticker(df, min_volume=None, max_price=None):
    """データフレームの銘柄コードから株価情報を取得して、指定された出来高や株価でフィルタリングする

    :param df: 対象のデータフレーム
    :param min_volume: 出来高の最小値を指定
    :param max_price: 株価の最大値を指定
    :return: 出来高や株価でフィルタリングしたデータフレーム
    """
    # 空のDataFrameを作成
    result_df = pd.DataFrame()

    # 各銘柄についてループ
    for index, row in df.iterrows():
        # コードを取得
        code = row['コード']

        # 株価情報を取得
        stock_data = get_stock_data_yapi2(code, period_type='day', period=1)

        # 出来高と価格の条件に合わない場合、次の銘柄コードに遷移
        volume = stock_data['Volume'][-1]
        price = stock_data['Close'][-1]
        if min_volume is not None and volume < min_volume:
            continue
        if max_price is not None and price > max_price:
            continue

        # 銘柄名と業種を設定
        filtered_df = pd.DataFrame({
            '銘柄名': [row['銘柄名']],
            'コード': [code],
            '業種': [row['業種']],
            '出来高': [volume],
            '株価': [price],
        }, index=[index])

        # 結果をresult_dfに追加
        result_df = pd.concat([result_df, filtered_df])

    return result_df

In [88]:
from pandas.io.formats.info import DataFrameTableBuilderNonVerbose
import requests
import pandas as pd

def get_ticker_jpx400(min_volume=None, max_price=None):
    """JPX400の銘柄コード一覧を取得する

    :param min_volume: 出来高の最小値を指定
    :param max_price: 株価の最大値を指定
    :return: JPX400の銘柄コード一覧
    """

    url = "https://indexes.nikkei.co.jp/nkave/archives/file/jpx_nikkei_index_400_weight_jp.csv"
    r = requests.get(url)
    with open('jpx_nikkei_index_400_weight_jp.csv', 'wb') as output:
        output.write(r.content)

    df = pd.read_csv("./jpx_nikkei_index_400_weight_jp.csv", encoding='shift_jis')
    # 不要なレコードとカラムを削除
    df = df.iloc[:-1]   # 最後の行は説明文のため除外
    df = df[['銘柄名', 'コード', '業種']]   # 必要なカラムのみにする
    df['コード'] = df['コード'].astype(int) # コードを整数に変更

    # 指定された引数でレコードをフィルタリングする
    if min_volume or max_price:
        df = filter_ticker(df, min_volume, max_price)

    return df

In [94]:
# 以下の条件に合致するJPX400の銘柄コードを取得
# 出来高　... 20万株以上
# 株価 ... 5000円以下
jpx400 = get_ticker_jpx400(min_volume=200000, max_price=5000)
jpx400

,銘柄名,コード,業種,出来高,株価
0,ニッスイ,1332,水産・農林業,1062600,596.0
1,ウエストホールディングス,1407,建設業,222400,3045.0
3,ミライト・ワン,1417,建設業,295400,1701.0
4,ＩＮＰＥＸ,1605,鉱業,7722300,1458.0
5,安藤・間,1719,建設業,353900,926.0
...,...,...,...,...,...
387,ＳＣＳＫ,9719,情報・通信業,652500,2132.0
389,メイテック,9744,サービス業,347400,2251.0
390,ＮＳＤ,9759,情報・通信業,345100,2585.0
395,ミスミグループ本社,9962,卸売業,842900,3380.0


In [95]:
get_stock_data_yapi2(jpx400.loc[jpx400['銘柄名'] == 'ニッスイ', 'コード'].values[0])

,Open,High,Low,Close,Volume
Date,,,,,
2013-05-13,198.0,206.0,197.0,206.0,4388400
2013-05-14,206.0,208.0,201.0,201.0,3033600
2013-05-15,203.0,222.0,203.0,219.0,10037200
2013-05-16,221.0,225.0,203.0,206.0,7419700
2013-05-17,212.0,219.0,210.0,219.0,5122400
...,...,...,...,...,...
2023-05-01,598.0,600.0,593.0,596.0,1474200
2023-05-02,598.0,601.0,591.0,596.0,2070200
2023-05-08,591.0,599.0,591.0,596.0,1811000
